<a href="https://colab.research.google.com/github/jpastolfo/Seismic-Velocity-Model-Inversion/blob/main/Sismico_Modelo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install deepwave==0.0.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 8.1 MB/s eta 0:00:00
  Created wheel for deepwave: filename=deepwave-0.0.10-py3-none-any.whl size=45121 sha256=80906b7265eec3d5f8b7cfb5c1944ce7fe68de546d9f0cfe0fead9e36d519279
  Stored in directory: /root/.cache/pip/wheels/53/54/e4/aca9daf6810805e0b08a4d4505e0fc4a9f6540f22e2bc9b308
Successfully built deepwave


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Gerador de Modelos
### Output - gerar primeiro para poder basear os nossos inputs

In [4]:
#gera modelos de sedimentos com deposição, dobras e falhamentos
#imports
import time
import scipy.ndimage
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import seaborn as sns
import cv2
import sys
import struct
from scipy import interpolate
from scipy.ndimage import gaussian_filter

In [5]:
#parametros
n_models=300

#tamanho dos modelos a serem gerados
nx=128
nz=128

#número de pacotes a serem depositados e número de camadas em cada pacote
npac=1
pac_size=12
nc=1+npac*pac_size
cam_min=2
cam_max=int((1.5*nz)/(npac*pac_size))

#eventos geológicos
dobra=False
erode=False
falha=False
diretorio='/content/drive/MyDrive/sismico/'

#intervalo de velocidades a serem inseridas
vmin=1550
vmax=3500
layer_inversion=True
wlayer=30
limita_vel=False

#parâmetros dos dobramentos
per_max1=nx/2
per_min1=nx/4
# per_max2=nx/20
# per_min2=nx/50
amp_min1=4
amp_max1=20
# amp_min2=8
# amp_max2=20
fase_min=0
fase_max=int(nx*np.pi)

In [6]:
for imod in range(1,n_models+1):
    print('indmod',imod)
    #inicializações
    nz_pac=nz
    ind_pac=0
    labels=np.zeros(shape=(nz,nx),dtype=int)
    d_max=nz

    #laço sobre o número de pacotes
    for i in range(0,npac):
        ind_pac=ind_pac+1
        if(ind_pac!=1):
            labels[0,:]=0
            labels[nz-1,:]=1
            d_max=np.amin(np.where(labels!=0)[0])
        #deposição das camadas    
        for cam in range(0,pac_size):
            # cam_num=nc-(cam+(ind_pac-1)*pac_size)
            cam_num=cam+1+(ind_pac-1)*pac_size
            # print('camn = ',cam_num)
            cam_thick=random.randint(cam_min,cam_max)
            d_max=d_max-cam_thick 
            #grava as interfaces
            for x in range(0,nx):
                auxlabels=np.zeros(shape=(nz),dtype=int)
                val=[]
                val=np.where(labels[:,x]==0)
                auxlabels[val]=cam_num
                labels[d_max:nz,x]=labels[d_max:nz,x]+auxlabels[d_max:nz]
        ########################################
        #dobramento das camadas#################
        if (dobra):
          # if(random.randint(0,1) == 1 or i==0):
          periodo=random.uniform(per_min1,per_max1)
          amp=random.uniform(amp_min1,amp_max1)
          # else:
          #     periodo=random.uniform(per_min2,per_max2) 
          #     amp=random.uniform(amp_min2,amp_max2)
          fase=random.uniform(fase_min,fase_max)
          labelsaux=np.zeros(shape=(nz,nx),dtype=int)
          # print('amp, periodo', amp,periodo)
          for x in range(0,nx):
              z_s=int((amp*np.sin(x/periodo+fase)))
              for z in range(0,nz):
                  labelsaux[z,x]=labels[np.clip(z+z_s,0,nz-1),x]
          labels=np.copy(labelsaux)
        labels[0,:]=0
        labels[nz-1,:]=1
        zmin=np.amin(np.where(labels!=0)[0])
        #########################################
        #introduz falhas no modelo################
        if (falha):
          escala=nz/nx
          n_falhas=1
          ang=random.uniform(np.pi/12,np.pi/6)
          sinal=random.randint(0,1)
          if sinal ==0:
              ang=(-1)*ang
          alfa=np.tan(ang)*escala
          for ii in range(0,n_falhas):
              rejeito=random.randint(10,30)
              sinal=random.randint(0,1)
              if sinal ==0:
                  rejeito=(-1)*rejeito
              pini=random.randint(50,nx-50)
              labelsaux=np.copy(labels)
              labels[0,:]=0
              labels[nz-1,:]=1
              zref=np.amin(np.where(labels[:,pini]!=0))
              if alfa <= 0:
                  for z in range(0,nz):
                      x_s=int((alfa*(zref-z)))
                      for x in range(np.clip(pini+x_s,0,nx-1),nx):
                          labelsaux[z,x]=labels[np.clip(z+rejeito,0,nz-1),np.clip(x-x_s,0,nx-1)]
                  labels=np.copy(labelsaux)
      
              if alfa > 0:
                  for z in range(0,nz):
                      x_s=int((alfa*(zref-z)))
                      for x in range(np.clip(pini+x_s,0,nx-1),nx):
                          labelsaux[z,x]=labels[np.clip(z+rejeito,0,nz-1),np.clip(x+x_s,0,nx-1)]
                  labels=np.copy(labelsaux)
        labels[0,:]=0
        labels[nz-1,:]=1
        zmin=np.amin(np.where(labels!=0)[0])
        ################################################
        #erosão- apaga as porções superiores do modelo
        if (erode):
          erosao=random.randint(0,1)
          if erosao == 1:
              perda=random.randint(10,80)
              labels[0:np.clip(zmin+perda,0,nz),:]=0
        #################################################
        labels[0,:]=0
        labels[nz-1,:]=1
        zmin=np.amin(np.where(labels!=0)[0])

    ############# fim do ciclo de deposição###################    
    last_label=np.amax(labels)
    #checa se a primeira camada não está muito funda###puxa todas as camadas para cima  
    if zmin >= (wlayer+10):
        labels[wlayer+5:wlayer+5+(nz-zmin),:]=labels[zmin:nz,:]
        labels[wlayer+5+(nz-zmin):nz,:]=1#coloca 1 na parte funda
        zmin=np.amin(np.where(labels!=0)[0])
   
    if zmin <= wlayer:#erode o sedimento até wlayer
        labels[0:wlayer,:]=0

    for x in range(0,nx): #define a última camada até wlayer
        auxlabels=np.zeros(shape=(nz),dtype=int)
        val=[]
        val=np.where(labels[:,x]==0)
        auxlabels[val]=auxlabels[val]+last_label+1
        labels[wlayer:nz,x]=labels[wlayer:nz,x]+auxlabels[wlayer:nz]

    last_label=np.amax(labels)

    #definir as velocidades que irão compor o modelo
    for cam in range(last_label):
        vels=np.linspace(vmin,vmax,last_label+1)
        vels_draw=[]

        if (layer_inversion==True):
            for i in range(0,len(vels)):
                x=np.random.normal(vels[i],(vels[1]-vels[0]))
                vels_draw.append(np.round(x,2))
        else:
            for i in range(0,len(vels)):
                x=np.random.normal(vels[i],(vels[1]-vels[0])/2)
                
                if (i==0):
                    while(x<vmin):
                        x=np.random.normal(vels[i],(vels[1]-vels[0])/2)
                        vels_draw.append(np.round(x,2))
                else:
                    check=0
                    while(vels_draw[i-1]>=x):
                        check=1
                        c_h=c_h+1
                        x=np.random.normal(vels[i],(vels[1]-vels[0])/2)
                        vels_draw.append(np.round(x,2))
    # adiciona camadas intermediárias com alta velocidade
    for i in range(len(vels_draw)):
        if(vels_draw[i] >= vmax):
                vels_draw[i]=vmax 
        alta=random.randint(0,9)
        if (alta > 6):
            vels_draw[i]=vels_draw[i]+(random.randint(1,5)*100)
            if(vels_draw[i] >= vmax):
                vels_draw[i]=vmax 

    #adiciona camadas intermediárias com baixa velocidade
    for i in range(len(vels_draw)):
        alta=random.randint(0,9)
        if (alta > 6):
            if(vels_draw[i] >= 2800):
                vels_draw[i]=vels_draw[i]-(random.randint(1,5)*100)

    ######gera o modelo de velocidades############
    last_label=np.amax(labels)
    modfinal = np.zeros(shape=(nz,nx))+1500
    maxval=np.amax(vels_draw)
    for z in range(wlayer,nz):
        for x in range(0,nx):
            modfinal[z,x]=vels_draw[int(np.clip(nc-labels[z,x],0,len(vels_draw)-1))]
            if (limita_vel):
              # esses dois if controlam os limites inferiores e superiores de velocidade
              if modfinal[z,x] >= maxval: 
                modfinal[z,x]=vmax
              if modfinal[z,x] <= vmin:
                modfinal[z,x]= vmin
    print(np.amin(modfinal))
    print(np.amax(modfinal))
      
    # modfinal.T.tofile(diretorio+'/Bin/model_{}.vp'.format(imod))
    np.save(diretorio+'Modelo1/Bin/model_{}.vp'.format(imod),modfinal)
    plt.imsave(diretorio+'Modelo1/Figs/teste_{}.jpeg'.format(imod),modfinal,cmap='jet',vmin=vmin, vmax=vmax)

indmod 1
1500.0
3271.39
indmod 2
1500.0
3493.89
indmod 3
1500.0
3426.92
indmod 4
1500.0
3500.0
indmod 5
1500.0
3500.0
indmod 6
1500.0
3455.75
indmod 7
1471.34
3109.52
indmod 8
1500.0
2899.95
indmod 9
1500.0
3500.0
indmod 10
1445.57
3500.0
indmod 11
1500.0
3500.0
indmod 12
1500.0
3500.0
indmod 13
1500.0
3300.0
indmod 14
1500.0
3269.57
indmod 15
1500.0
3243.26
indmod 16
1500.0
3428.56
indmod 17
1498.7
3500.0
indmod 18
1500.0
3500.0
indmod 19
1500.0
3462.27
indmod 20
1395.87
3210.13
indmod 21
1500.0
3500.0
indmod 22
1500.0
3467.98
indmod 23
1500.0
3500.0
indmod 24
1432.22
3313.49
indmod 25
1500.0
3242.14
indmod 26
1500.0
3500.0
indmod 27
1500.0
3500.0
indmod 28
1500.0
3500.0
indmod 29
1500.0
3500.0
indmod 30
1500.0
3343.99
indmod 31
1500.0
3458.84
indmod 32
1500.0
3385.35
indmod 33
1500.0
3500.0
indmod 34
1500.0
2987.38
indmod 35
1500.0
3300.0
indmod 36
1500.0
3300.0
indmod 37
1500.0
3500.0
indmod 38
1251.17
3500.0
indmod 39
1500.0
3500.0
indmod 40
1500.0
3500.0
indmod 41
1500.0
3498.06
i

# Gerador de Shots
### input - apesar de ser input, eles são gerados a partir dos modelos!

In [7]:
import torch
import scipy.ndimage
import matplotlib.pyplot as plt
import deepwave
from deepwave import scalar

In [8]:
#Determinando inicialmente as variáveis dos shots que serão gerados
dx = 10.0 # 10m
dt = 0.0030 # 1ms
nz = 128
ny = 128
nt = 640# 1.92s de simulação
peak_freq = 8
peak_time = 1.0 / peak_freq
num_shots = 12
num_sources_per_shot = 1
num_receivers_per_shot = 128 # um receptor a cada cela do modelo
source_spacing = 10 # em número de celas (100m)
receiver_spacing = 1 # em número de celas (10m)
first_source= 4 # borda para acomodar os 12 tiros
first_receiver=0
source_depth=1
receiver_depth=1

In [9]:
device = torch.device('cuda:0') # rodar a placa (retirar depois)

In [10]:
#geometria do levantamento
# source_locations
source_locations = torch.zeros(num_shots,num_sources_per_shot,2,
                               dtype=torch.long, device=device)
source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(num_shots)*source_spacing+ first_source

# receiver_locations
receiver_locations = torch.zeros(num_shots,num_receivers_per_shot, 2,
                                 dtype=torch.long, device=device)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = ((torch.arange(num_receivers_per_shot)*receiver_spacing + first_receiver)
    .repeat(num_shots, 1))

# source_amplitudes
source_amplitudes = (deepwave.wavelets.ricker(peak_freq, nt, dt,peak_time)
    .repeat(num_shots,num_sources_per_shot, 1)
    .to(device))

In [11]:
#definições dos modelos
model_homo = torch.ones(ny, nz) * 1500.0 # modelo da água para excluir a onda direta
model_homogenous=model_homo.to(device)
# Propagate homogeneous 
out = scalar(model_homogenous,dx,dt,source_amplitudes=source_amplitudes,
             source_locations=source_locations,
             receiver_locations=receiver_locations,
             accuracy=8,
             pml_freq=peak_freq,pml_width=[50,50,50,50])
receiver_amplitudes_homo=out[-1] #copia as amplitudes do dado de tiro

In [14]:
for nummodel in range(1,301):
#carga modelo
  model=np.load(diretorio+"Modelo1/Bin/model_{}.vp.npy".format(nummodel))
  model_npy=(model.T).astype(np.float32) #rede está desenhada para esse tamanho de problema
  model =torch.ones(ny, nz) * model_npy 
  model_prop=model.to(device)
  # Propagate
  out = scalar(model_prop,dx,dt,source_amplitudes=source_amplitudes,
              source_locations=source_locations,
              receiver_locations=receiver_locations,
              accuracy=8,
              pml_freq=peak_freq,pml_width=[50,50,50,50])
  receiver_amplitudes=out[-1] #copia as amplitudes do dado de tiro
  np.save(diretorio+'Shots1/shot_{}.npy'.format(nummodel),(receiver_amplitudes-receiver_amplitudes_homo).cpu())
  print(nummodel)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
